In [1]:
from ipywidgets import interactive, interact
import ipywidgets as widgets
from IPython.display import SVG, display
from matplotlib import colormaps

import json
import math
import chess

In [2]:
cmap = colormaps["PuRd"]

def hexify(f):
    assert f >= 0 and f <= 1
    return f"{int(f * 255):02x}"

def get_color(val):
    if val < 1e-3:
        return f"#ffffff77"
    r, g, b, _ = cmap(val)
    return f"#{hexify(r)}{hexify(g)}{hexify(b)}ff"

def replay(trace, step):
    steps = trace["steps"]
    b = chess.Board()
    for move in steps[0:step]:
        b.push(chess.Move.from_uci(move[0]))

    moves, num_acts, q_values = zip(*steps[step][2])
    num_acts = [n for _, n, _ in steps[step][2]]
    q_values = [v for _, _, v in steps[step][2]]
    sum_num = sum(num_acts) + 1e-4
    score = [v / sum_num for v in num_acts]
    return b, list(zip(moves, score, q_values, num_acts))

In [3]:
filename = "../runs/97/trace2.json"
# filename = "../debug-traces/trace4.json"
# filename = "../trace6.json"
# filename = "../replay.json"

In [4]:
with open(filename) as f:
    trace = json.load(f)

outcome = trace["outcome"]
steps = trace["steps"]
    
def f(step=widgets.IntSlider(min=0, max=len(steps), step=1, value=0)):
    b = chess.Board()
    for move in steps[0:step]:
        b.push(chess.Move.from_uci(move[0]))

    if step == len(steps):
        return b

    moves = [m for m, _, _ in steps[step][2]]
    num_acts = [n for _, n, _ in steps[step][2]]
    sum_num = sum(num_acts) + 1e-4
    score = [v / sum_num for v in num_acts]
    distr = sorted(zip(moves, score), key=lambda p: p[1], reverse=True)
    labels = widgets.HBox([
        widgets.Label(value=f"{m}", style=dict(background=get_color(c)))
        for m, c in distr
    ])
    out = widgets.Output()
    with out:
        display(SVG(data=b._repr_svg_()))
    return widgets.VBox([out, labels])

interact(f);

interactive(children=(IntSlider(value=0, description='step', max=36), Output()), _dom_classes=('widget-interac…

In [122]:
outcome

{'termination': 'Checkmate', 'winner': 'Black'}

In [10]:
from importlib import reload
import train
import libencoder
reload(train)
reload(libencoder)

<module 'libencoder' from '/home2/jiasen/workspace/smart-chess-rust/target/release/libencoder.so'>

In [11]:
import torch
import nn
model = nn.load_model(
    device="cpu", 
    checkpoint="../runs/96/tb_logs/chess/version_0/last.ckpt"
)

model1 = nn.load_model(device="cpu")

..loading checkpoint:  ../runs/96/tb_logs/chess/version_0/last.ckpt


In [12]:
def infer(fn, index):
    ds = train.ChessDataset(fn)

    board_enc = ds[index][0]
    moves_enc = ds.steps[index][3]
  
    pi, value = model(board_enc.unsqueeze(0))
    pi = torch.exp(pi.detach()).squeeze()

    pi = pi[moves_enc]
    pi = pi / pi.sum()

    value = value.detach().squeeze()
    return pi, value

In [29]:
with open(filename) as f:
    trace = json.load(f)

step = 35
prior, value = infer(filename, step)
total_n = sum([n for _, n, _ in trace["steps"][step][2]])

distr = replay(trace, step)

In [30]:
total_n, prior, value

(162,
 tensor([0.1056, 0.0449, 0.0400, 0.0685, 0.1050, 0.0829, 0.0829, 0.0340, 0.0125,
         0.0047, 0.0041, 0.0084, 0.0087, 0.0022, 0.0345, 0.0160, 0.0009, 0.0006,
         0.0007, 0.0011, 0.0002, 0.0033, 0.0046, 0.0004, 0.0071, 0.0483, 0.0315,
         0.0535, 0.0862, 0.0386, 0.0680]),
 tensor(-0.8117))

In [24]:
def uct(prior, total_n, q_value, current_n, reverse, cpuct):
    award = q_value / (current_n + 1e-5) * (-1 if reverse else 1)
    exploration = math.sqrt(total_n) / (1 + current_n) * prior * cpuct
    return (f"{award + exploration:0.3f}", f"{award:0.3f}", f"{exploration:0.3f}")

In [25]:
uct(0.052366, 160, -0.043745, 1, False, 0.2)

('0.022', '-0.044', '0.066')

In [26]:
uct(0.040944, 160, 0.087995, 1, False, 0.2)

('0.140', '0.088', '0.052')

In [31]:
rotate = step % 2 == 1
import pandas as pd
pd.DataFrame(
    [(m, n, q, prior[i].item(), uct(prior[i].item(), total_n, q, n, rotate, 5)) for i, (m, n, q) in enumerate(trace["steps"][step][2])],
    columns=("move", "nact", "q", "prior", "uct"),
)

,move,nact,q,prior,uct
0,h8h7,5,4.406776,0.105646,"(0.239, -0.881, 1.121)"
1,h8h6,5,4.406776,0.044919,"(-0.405, -0.881, 0.476)"
2,g8h6,5,4.278743,0.039956,"(-0.432, -0.856, 0.424)"
3,g8f6,6,5.138655,0.068519,"(-0.234, -0.856, 0.623)"
4,a8a7,6,5.138655,0.104965,"(0.098, -0.856, 0.954)"
5,d7e8,5,4.508484,0.082861,"(-0.023, -0.902, 0.879)"
6,d7d8,6,3.692343,0.082861,"(0.138, -0.615, 0.753)"
7,d7c8,3,2.532134,0.034025,"(-0.303, -0.844, 0.541)"
8,e6f5,4,3.493754,0.012503,"(-0.714, -0.873, 0.159)"
9,e6g4,2,1.878221,0.004723,"(-0.839, -0.939, 0.100)"


In [13]:
data = [(chess.Move.from_uci(step[0]), [n[1]for n in step[2]]) for step in trace["steps"]]

# step16 = [0] * 22
# step16[4] = 1
# ret = libencoder.encode(data[:16] + [(chess.Move.from_uci("b5c3"), step16)])
ret = libencoder.encode(data)
eb, em, ed, mi = ret[22]

inp = np.concatenate((eb, em), axis=-1).astype(np.float32)
inp = inp.transpose((2, 0, 1))

inp = torch.from_numpy(inp).unsqueeze(0)
pi, value = model(inp)
pi = torch.exp(pi.detach()).squeeze()
pi = pi[mi]
pi = pi / pi.sum()
value = value.detach().squeeze()
pi, value

NameError: name 'np' is not defined

In [9]:
data = [(chess.Move.from_uci(step[0]), [n[1]for n in step[2]]) for step in trace["steps"]]
ret = libencoder.encode_steps(data)
eb, em, ed, mi = ret[35]
em[0]

RS 1 0 0 [1 0] 0
RS 1 7 0
RS 1 0 0 [1 0] 0
RS 1 6 0
RS 1 0 0 [1 0] 0
RS 1 5 0
RS 1 0 0 [1 0] 0
RS 1 4 0
RS 1 0 0 [1 0] 0
RS 1 3 0
RS 1 0 0 [1 0] 0
RS 1 2 0
RS 1 0 0 [1 0] 0
RS 1 1 0
RS 1 0 0 [1 0] 0
RS 1 0 0
RS 2 0 1 [1 0] 0
RS 1 7 1
RS 2 0 1 [1 0] 0
RS 1 6 1
RS 2 0 1 [1 0] 0
RS 1 5 1
RS 2 0 1 [1 0] 0
RS 1 4 1
RS 2 0 1 [1 0] 0
RS 1 3 1
RS 2 0 1 [1 0] 0
RS 1 2 1
RS 2 0 1 [1 0] 0
RS 1 1 1
RS 2 0 1 [1 0] 0
RS 1 0 1
RS 1 0 0 [1 0] 0
RS 1 0 0
RS 1 0 0 [1 0] 0
RS 1 1 0
RS 1 0 0 [1 0] 0
RS 1 2 0
RS 1 0 0 [1 0] 0
RS 1 3 0
RS 1 0 0 [1 0] 0
RS 1 4 0
RS 1 0 0 [1 0] 0
RS 1 5 0
RS 1 0 0 [1 0] 0
RS 1 6 0
RS 1 0 0 [1 0] 0
RS 1 7 0
RS 2 0 1 [1 0] 0
RS 1 0 1
RS 2 0 1 [1 0] 0
RS 1 1 1
RS 2 0 1 [1 0] 0
RS 1 2 1
RS 2 0 1 [1 0] 0
RS 1 3 1
RS 2 0 1 [1 0] 0
RS 1 4 1
RS 2 0 1 [1 0] 0
RS 1 5 1
RS 2 0 1 [1 0] 0
RS 1 6 1
RS 2 0 1 [1 0] 0
RS 1 7 1
RS 1 -1 0 [ 1 -1] 7
RS 0 4 49
RS 1 0 0 [1 0] 0
RS 0 3 0
RS 5 5 4 [1 1] 1
RS 0 2 11
RS 4 4 3 [1 1] 1
RS 0 2 10
RS 3 3 2 [1 1] 1
RS 0 2 9
RS 2 2 1 [1 1] 1
RS 0 2 8
RS 1 1

array([[ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1],
       [ 0, 18,  0,  0,  0,  0,  1]], dtype=uint32)

In [34]:
eb.shape

(8, 8, 112)

In [36]:
ed.shape

(4672,)

In [10]:
mi

[0,
 1,
 136,
 129,
 511,
 911,
 904,
 897,
 1436,
 1437,
 1562,
 1561,
 1554,
 1575,
 1533,
 1540,
 1534,
 2665,
 2664,
 2663,
 2649,
 2677,
 2635,
 2636,
 1613,
 1728,
 1759,
 2051,
 730,
 1752,
 731]

In [11]:
b, r = replay(trace, 35)

In [12]:
expected_moves = list(b.legal_moves)

In [6]:
import sys
sys.path.append("/home2/jiasen/workspace/smart-chess/")

In [7]:
import chess
import libencoder
import encode
import utils

In [13]:
[(a == b, a, b) for a, b in zip([encode.MoveEncoding.decode(chess.BLACK, c)[0] for c in mi], expected_moves)]

[(True, Move.from_uci('h8h7'), Move.from_uci('h8h7')),
 (True, Move.from_uci('h8h6'), Move.from_uci('h8h6')),
 (True, Move.from_uci('g8h6'), Move.from_uci('g8h6')),
 (True, Move.from_uci('g8f6'), Move.from_uci('g8f6')),
 (True, Move.from_uci('a8a7'), Move.from_uci('a8a7')),
 (True, Move.from_uci('d7e8'), Move.from_uci('d7e8')),
 (True, Move.from_uci('d7d8'), Move.from_uci('d7d8')),
 (True, Move.from_uci('d7c8'), Move.from_uci('d7c8')),
 (True, Move.from_uci('e6f5'), Move.from_uci('e6f5')),
 (True, Move.from_uci('e6g4'), Move.from_uci('e6g4')),
 (True, Move.from_uci('c6c8'), Move.from_uci('c6c8')),
 (True, Move.from_uci('c6c7'), Move.from_uci('c6c7')),
 (True, Move.from_uci('c6b7'), Move.from_uci('c6b7')),
 (True, Move.from_uci('c6d6'), Move.from_uci('c6d6')),
 (True, Move.from_uci('c6c5'), Move.from_uci('c6c5')),
 (True, Move.from_uci('c6b5'), Move.from_uci('c6b5')),
 (True, Move.from_uci('c6c4'), Move.from_uci('c6c4')),
 (True, Move.from_uci('d4g7'), Move.from_uci('d4g7')),
 (True, Mo

In [3]:
m = chess.Move.from_uci("e6f5")
encode.MoveEncoding.encode(chess.BLACK, m), libencoder.encode_move(chess.BLACK, m)

PY 1 -1 0 <object type:list(int64)<iv=None>> 7
PY 2 3 49
RS 1 -1 0 [ 1 -1] 6
RS 2 3 42


(1436, 1429)

In [4]:
mr = utils.rotate(m)
encode.MoveEncoding.encode(chess.WHITE, mr), libencoder.encode_move(chess.WHITE, mr)

PY 0 <object type:UniTuple(int64 x 2)> 7
PY 2 3 49
RS 0 [ 1 -1] 6
RS 2 3 42


(1436, 1429)

In [5]:
encode.MoveEncoding.encode(chess.WHITE, chess.Move.from_uci("e6f5")), libencoder.encode_move(chess.WHITE, chess.Move.from_uci("e6f5"))

true, true, 1, 0


(3233, 3233)

In [6]:
mr

Move.from_uci('d3c4')

In [19]:
import queenmoves
import knightmoves
import underpromotions

In [20]:
queenmoves.encode(mr)

1436